Analysis to get a baseline using **all features** (FER, pupil and GSR) without feature selection.

In [ ]:
# IMPORTS
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score, root_mean_squared_error
from scipy.stats import pearsonr
from lightgbm import LGBMRegressor
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
from sklearn.base import clone
from copy import deepcopy
from sklearn.model_selection import GridSearchCV, GroupKFold
from lightgbm import LGBMRegressor

In [3]:
# Defining the directories
full_video_directory = r"D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Features-NEW/Full"
intervals_directory = r"D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Features-NEW/Intervals"
ground_truth_directory = r"D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Ground truth"
results_directory = r"D:/MASTER/Uni of Essex/Disseration/Data-Multimotion/Results"

In [ ]:
# Full FER - loading the csv
df_fer_full = pd.read_csv(os.path.join(full_video_directory, "fer_features.csv"))

# Intervals FER - loading the csv
df_fer_intervals = pd.read_csv(os.path.join(intervals_directory, "fer_features_intervals.csv"))

# Merging the intervals and full FER features
df_fer = pd.merge(df_fer_full, df_fer_intervals, on=['participant', 'video'],
                               suffixes=('_whole', '_interval'))


# Full Pupil - loading the csv
df_pupil_full = pd.read_csv(os.path.join(full_video_directory, "Not_Interval_60_part_all_stat_features_12062025.csv"))
df_pupil_full.rename(columns={"Participant": "participant", "simuli_name_1": "video"}, inplace=True)
df_pupil_full.drop(columns=['Arousal', 'Valence', 'simuli_name_2','Unnamed: 0'], inplace=True)

# Intervals - Loading the csv
df_pupil_interval = pd.read_csv(os.path.join(intervals_directory, "Interval_60_part_all_stat_features_01062025.csv"))
df_pupil_interval.rename(columns={"Participant": "participant", "simuli_name_1": "video"}, inplace=True)
df_pupil_interval.drop(columns=['Arousal', 'Valence', 'simuli_name_2', 'Unnamed: 0'], inplace=True)

# Merging the intervals and full Pupil features
df_pupil = pd.merge(df_pupil_full, df_pupil_interval, on=['participant', 'video'],
                    suffixes=('_whole', '_interval'))


# Full GSR - Loading the csv
df_gsr_full = pd.read_csv(os.path.join(full_video_directory, "gsr_features.csv"))
df_gsr_full.rename(columns={"ParticipantID": "participant", "StimulusName": "video"}, inplace=True)

# Intervals GSR - Loading the csv
df_gsr_intervals = pd.read_csv(os.path.join(intervals_directory, "gsr_features_intervals.csv"))
df_gsr_intervals.rename(columns={"ParticipantID": "participant", "StimulusName_0": "video"}, inplace=True)

# Merging the intervals and full GSR features
df_gsr = pd.merge(df_gsr_full, df_gsr_intervals, on=['participant', 'video'],
                  suffixes=('_whole', '_interval'))


participant_col = 'participant'
video_col = 'video'

fer_features = [col for col in df_fer.columns if col not in [participant_col, video_col]]
pupil_features = [col for col in df_pupil.columns if col not in [participant_col, video_col]]
gsr_features = [col for col in df_gsr.columns if col not in [participant_col, video_col]]


# Merge df_fer and df_pupil first
df_merged = pd.merge(df_fer, df_pupil, on=['participant', 'video'], how='inner')

# Then merge with df_gsr
df = pd.merge(df_merged, df_gsr, on=['participant', 'video'], how='inner')

In [ ]:
# Checking the number of participants 

print('Participants FER:', len(df_fer['participant'].unique()))
print('Participants Pupil:', len(df_pupil['participant'].unique()))
print('Participants GSR:', len(df_gsr['participant'].unique()))
print('Participants merged:', len(df['participant'].unique()))

Participants FER: 50
Participants Pupil: 59
Participants GSR: 59
Participants merged: 45


In [ ]:
# Get columns with NaNs and sort by number of NaNs
nan_summary = df.isnull().sum()
nan_summary = nan_summary[nan_summary > 0].sort_values(ascending=False)

# Display columns with NaNs
print(nan_summary.head(100))

# Get column names with NaNs
missing_cols = nan_summary.index.tolist()

# Impute missing values with the mean
imputer = SimpleImputer(strategy='mean')  
df[missing_cols] = imputer.fit_transform(df[missing_cols])

corr_kurtosis_whole       1301
corr_skewness_whole       1301
corr_auc_whole            1301
diff_kurtosis_whole       1301
diff_skewness_whole       1301
diff_auc_whole            1301
corr_kurtosis_interval     537
corr_skewness_interval     537
corr_auc_interval          537
diff_kurtosis_interval     537
diff_skewness_interval     537
diff_auc_interval          537
dtype: int64


**Ground truth**

In [ ]:
# Load full ground truth (this is done only once)
gt_full_set = pd.read_csv(os.path.join(ground_truth_directory, "individual_ground_truth.csv"))

# Renaming so it is the same as FER and Pupil
gt_full_set.rename(columns={"Participant": "participant", "Stimulus_Name": "video"}, inplace=True)

# Function to load ground truth file for excluding one participant for Leave One Participant Out
def load_ground_truth_exclude(participant_id):
    # Construct the file path
    file_path = os.path.join(
        ground_truth_directory,
        f"Leave-one-out/individual_ground_truth_no_{participant_id}.csv"
    )

    try:
        gt_df = pd.read_csv(file_path)
        return gt_df
    except FileNotFoundError:
        print(f"[Warning] Ground truth file not found for participant {participant_id} at: {file_path}")
        return None 

In [ ]:
def get_train_test_data(df, gt_full_set, test_pid, target):
    """Split data in train and test and merge with correct ground truth."""
    test_data = df[df["participant"] == test_pid].copy()
    test_gt = gt_full_set[gt_full_set["participant"] == test_pid]
    test_data = test_data.merge(
    test_gt[["participant", "video", target]],
      on=["participant", "video"],
          how="inner",
      )
    train_data = df[df["participant"] != test_pid].copy()
    pid_gt = load_ground_truth_exclude(test_pid)

    if pid_gt is not None:
      pid_gt.rename(
            columns={"Participant": "participant", "Stimulus_Name": "video"}, inplace=True
        )

      train_data = train_data.merge(
            pid_gt[["participant", "video", target]],
            on=["participant", "video"],
            how="inner",
        )

      return train_data, test_data
    else:
      return None, None

In [9]:
def evaluate_model(y_true, y_pred):
    """Calculate evaluation metrics."""
    rmse = root_mean_squared_error(y_true, y_pred)
    nrmse = rmse / (y_true.max() - y_true.min())
    r2 = r2_score(y_true, y_pred)
    corr, p_val = pearsonr(y_true, y_pred)
    return {"NRMSE": nrmse, "R2": r2, "corr": corr, "p": p_val}


def print_summary(results_df):
    filtered = results_df
    summary_filtered = filtered[['NRMSE', 'R2', 'corr', 'p']].agg(['mean', 'std'])
    print("\n=== Summary (Including Outliers) ===")
    print(summary_filtered.round(4))
    excluded = results_df[~results_df.index.isin(filtered.index)]
    print("\nExcluded participants:")
    print(excluded[['participant', 'NRMSE', 'R2', 'corr']])

def print_filtered_summary(results_df):
    filtered = results_df[
        (results_df['R2'] > -1.0) &                 # Removes models with extremely poor fit
        (results_df['NRMSE'] < 1.5) &              # Removes predictions with very high error
        (results_df['corr'].abs() > 0.2)           # Keeps only modestly correlated results
    ]
    summary_filtered = filtered[['NRMSE', 'R2', 'corr', 'p']].agg(['mean', 'std'])
    print("\n=== Summary (Excluding Outliers) ===")
    print(f"\nFiltered out {len(results_df) - len(filtered)} out of {len(results_df)} participants.")
    print(summary_filtered.round(4))
    excluded = results_df[~results_df.index.isin(filtered.index)]
    print("\nExcluded participants:")
    print(excluded[['participant', 'NRMSE', 'R2', 'corr']])

# AROUSAL

Defining the method for LOPO, sequential feature selection and hyperparameter tuning is added but for the baseline, these will not be used.

In [ ]:
def run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls,
    model_kwargs={},
    grid_search=False,
    param_grid={},
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=None,
):
    """
    Runs LOPO cross-validation for arousal prediction 
    with optional Sequential Feature Selection and hyperparameter tuning.

    Supports evaluating different models with or without grid search and 
    feature selection, returning performance metrics across participants.
    
    """
    participants = df["participant"].unique()
    participant_results = []
    all_preds, all_true_values, all_test_participants = [], [], []
    
    # Grid Search
    best_score = -float("inf")
    best_estimator = None
    
    # Define inner CV
    inner_cv = GroupKFold(n_splits=5)
    if grid_search:
        for test_pid in participants:
            train_data, test_data = get_train_test_data(df, gt_full_set, test_pid, "Arousal")
            if train_data is None or test_data is None:
                continue
    
            X_train = train_data.drop(columns=["participant", "video", "Arousal"])
            y_train = train_data["Arousal"].values
    
            groups_train = train_data["participant"]
            grid = GridSearchCV(
                LGBMRegressor(),
                param_grid,
                cv=inner_cv.split(X_train, y_train, groups_train),
                scoring='r2',
                n_jobs=-1
            )
            grid.fit(X_train, y_train, groups=groups_train)
    
            if grid.best_score_ > best_score:
                best_score = grid.best_score_
                best_estimator = grid.best_estimator_
    
        model_cls = best_estimator
        model_kwargs = best_estimator.get_params()
        
    for test_pid in participants:
        train_data, test_data = get_train_test_data(df, gt_full_set, test_pid, "Arousal")
        if train_data is None or test_data is None:
            continue

        # Extract features and labels
        X_train = train_data.drop(columns=["participant", "video", "Arousal"])
        y_train = train_data["Arousal"].values
        X_test = test_data.drop(columns=["participant", "video", "Arousal"])
        y_test = test_data["Arousal"].values

        # Sequential Forward Selection (optional)
        if feature_selection:
            available_features = list(X_train.columns)
            selected_features = []
            best_score = -np.inf
            improved = True

            while improved and available_features:
                scores = []
                for feat in available_features:
                    try_features = selected_features + [feat]
                    estimator = clone(sfs_estimator_cls())
                    estimator.fit(X_train[try_features], y_train)
                    preds = estimator.predict(X_train[try_features])
                    score = scoring_func(y_train, preds)
                    scores.append((score, feat))
                scores.sort(reverse=True, key=lambda x: x[0])
                top_score, top_feature = scores[0]

                if top_score > best_score:
                    selected_features.append(top_feature)
                    available_features.remove(top_feature)
                    best_score = top_score
                    if max_features and len(selected_features) >= max_features:
                        break
                else:
                    improved = False

            X_train = X_train[selected_features]
            X_test = X_test[selected_features]

        # Train the final model
        model = model_cls(**deepcopy(model_kwargs))
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        # Evaluate
        fusion_metrics = evaluate_model(y_test, preds)
        fusion_metrics["participant"] = test_pid

        participant_results.append(fusion_metrics)
        all_preds.extend(preds)
        all_true_values.extend(y_test)
        all_test_participants.extend([test_pid] * len(y_test))

    return participant_results, all_preds, all_true_values, all_test_participants


**Linear Regression**

In [36]:
print("Linear Regression --------------- Arousal")

model_cls = LinearRegression
model_kwargs = {}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=40
)
results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

Linear Regression --------------- Arousal
       NRMSE         R2      corr             p participant
0   0.217893   0.618553  0.870690  5.142021e-12       F5tXL
1   0.138095   0.846784  0.932833  1.225714e-16        Gax8
2   0.222741   0.601389  0.892671  2.580932e-13       Hx7dO
3   0.177177   0.747789  0.897125  1.300662e-13       wohkw
4   0.196302   0.690403  0.873342  3.693082e-12       CVgs2
5   0.214205   0.636797  0.915332  1.379926e-14       G4dLl
6   0.189624   0.720169  0.910549  4.885029e-13       GFGzP
7   0.176830   0.748777  0.883733  9.363801e-13       PPjCX
8   0.175131   0.753581  0.926165  5.815981e-16       tMGNS
9   0.146017   0.828702  0.911430  1.142314e-14       5KB3V
10  0.223042   0.600315  0.865932  9.147837e-12       LR96S
11  0.154898   0.807230  0.903198  4.850837e-14        gf5X
12  0.204899   0.662693  0.852949  3.965785e-11       gjGzb
13  0.153894   0.809721  0.904979  3.587430e-14      A5feTy
14  0.197442   0.686797  0.861063  1.613307e-11       CoQm

In [64]:
print("Linear Regression --------------- Arousal")

model_cls = LinearRegression
model_kwargs = {}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,
    feature_selection=True,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=40
)
results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

Linear Regression --------------- Arousal
       NRMSE        R2      corr             p participant
0   0.228500  0.580513  0.844703  9.378031e-11       F5tXL
1   0.166062  0.778441  0.890249  3.699804e-13        Gax8
2   0.192057  0.703648  0.867033  8.022074e-12       Hx7dO
3   0.208040  0.652271  0.877314  2.217849e-12       wohkw
4   0.197721  0.685909  0.838920  1.665798e-10       CVgs2
5   0.248696  0.510416  0.856893  5.075594e-11       G4dLl
6   0.204454  0.674688  0.880264  3.147377e-11       GFGzP
7   0.174798  0.754517  0.874204  3.311147e-12       PPjCX
8   0.203620  0.666890  0.878002  2.026817e-12       tMGNS
9   0.182383  0.732751  0.866403  8.650151e-12       5KB3V
10  0.228311  0.581206  0.808866  2.385695e-09       LR96S
11  0.171169  0.764606  0.878466  1.906855e-12        gf5X
12  0.214481  0.630408  0.811653  1.901819e-09       gjGzb
13  0.174241  0.756080  0.877758  2.092745e-12      A5feTy
14  0.231449  0.569615  0.794269  7.386119e-09       CoQmx
15  0.208284  

**Ridge Regression**

In [37]:
print("Ridge Regression --------------- Arousal")

model_cls = Ridge

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=40
)
results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)


Ridge Regression --------------- Arousal


D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.387e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.38961e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.39405e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.44555e-21): result may not be accurate.
  return linalg

       NRMSE        R2      corr             p participant
0   0.239906  0.537587  0.864780  1.048297e-11       F5tXL
1   0.135989  0.851421  0.931502  1.693479e-16        Gax8
2   0.191270  0.706073  0.904045  4.205249e-14       Hx7dO
3   0.183489  0.729501  0.896786  1.371801e-13       wohkw
4   0.172788  0.760132  0.885016  7.833211e-13       CVgs2
5   0.229624  0.582628  0.898160  2.548326e-13       G4dLl
6   0.208874  0.660468  0.898624  2.941739e-12       GFGzP
7   0.168083  0.773016  0.882452  1.116636e-12       PPjCX
8   0.174984  0.753996  0.917688  3.453188e-15       tMGNS
9   0.148090  0.823803  0.911436  1.141076e-14       5KB3V
10  0.235556  0.554208  0.871025  4.933372e-12       LR96S
11  0.153342  0.811083  0.903926  4.290691e-14        gf5X
12  0.188146  0.715595  0.868712  6.551253e-12       gjGzb
13  0.151745  0.814999  0.916336  4.506742e-15      A5feTy
14  0.196407  0.690072  0.851362  4.698915e-11       CoQmx
15  0.173776  0.757379  0.877275  2.229217e-12      G5Xz

D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.38278e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.3882e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.37702e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.38648e-21): result may not be accurate.
  return linal

**Gradient Boosting Regressor**

In [38]:
print("GradientBoostingRegressor --------------- Arousal")

model_cls = GradientBoostingRegressor
model_kwargs = {}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=40
)

results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

GradientBoostingRegressor --------------- Arousal
       NRMSE        R2      corr             p participant
0   0.185209  0.724405  0.872445  4.133813e-12       F5tXL
1   0.090124  0.934742  0.984489  2.754783e-27        Gax8
2   0.089551  0.935569  0.985077  1.435130e-27       Hx7dO
3   0.133718  0.856342  0.971448  7.980557e-23       wohkw
4   0.084794  0.942233  0.970800  1.162832e-22       CVgs2
5   0.182088  0.737548  0.940763  4.609532e-17       G4dLl
6   0.093062  0.932601  0.976519  1.474538e-21       GFGzP
7   0.077138  0.952194  0.976215  3.707767e-24       PPjCX
8   0.113120  0.897192  0.974556  1.152197e-23       tMGNS
9   0.102090  0.916264  0.964361  3.275559e-21       5KB3V
10  0.126550  0.871332  0.971506  7.715194e-23       LR96S
11  0.116349  0.891240  0.946882  2.530186e-18        gf5X
12  0.151640  0.815253  0.962920  6.356796e-21       gjGzb
13  0.102039  0.916347  0.967499  7.002750e-22      A5feTy
14  0.135387  0.852734  0.928114  3.746767e-16       CoQmx
15  0.

**SVR**

In [40]:
print("SVR -------------- Arousal")

model_cls = SVR
model_kwargs = {'kernel':'rbf', 'C':1.0, 'epsilon':0.1}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,
    model_kwargs=model_kwargs,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=40
)

results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

SVR -------------- Arousal
       NRMSE        R2      corr         p participant
0   0.365046 -0.070636 -0.002454  0.988667       F5tXL
1   0.355729 -0.016686 -0.081268  0.637510        Gax8
2   0.356304 -0.019974  0.289744  0.086511       Hx7dO
3   0.354049 -0.007102 -0.041514  0.810024       wohkw
4   0.358967 -0.035276 -0.142910  0.405704       CVgs2
5   0.369835 -0.082690  0.088789  0.612008       G4dLl
6   0.361512 -0.017081 -0.120221  0.512212       GFGzP
7   0.357252 -0.025410  0.151212  0.378680       PPjCX
8   0.361109 -0.047666  0.077765  0.652131       tMGNS
9   0.356755 -0.022558 -0.022930  0.894396       5KB3V
10  0.363798 -0.063329  0.052695  0.760194       LR96S
11  0.356871 -0.023223 -0.195895  0.252200        gf5X
12  0.356396 -0.020498 -0.064896  0.706893       gjGzb
13  0.355667 -0.016327  0.052557  0.760806      A5feTy
14  0.358662 -0.033515 -0.130863  0.446807       CoQmx
15  0.356079 -0.018686  0.059273  0.731307      G5XzoP
16  0.372733 -0.116203 -0.164638  0.33

**LGBM**

In [41]:
print("LGBM -------------- Arousal")
model_cls = LGBMRegressor
model_kwargs = {'num_leaves':63, 'max_depth':10, 'learning_rate':0.1, 'n_estimators':200}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,  # now a callable that returns a fitted model
    model_kwargs=model_kwargs,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,  # optional
    scoring_func=r2_score,
    max_features=40
)
results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

LGBM -------------- Arousal
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003784 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 85191
[LightGBM] [Info] Number of data points in the train set: 1575, number of used features: 344
[LightGBM] [Info] Start training from score 0.014041
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

# VALENCE

Defining the method for LOPO, sequential feature selection and hyperparameter tuning is added but for the baseline, these will not be used.

In [ ]:
def run_participant_loop_sfs_valence(
    df,
    gt_full_set,
    model_cls,
    model_kwargs={},
    grid_search=False,
    param_grid={},
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=None,
):
    """
    Runs LOPO cross-validation for arousal prediction 
    with optional Sequential Feature Selection and hyperparameter tuning.

    Supports evaluating different models with or without grid search and 
    feature selection, returning performance metrics across participants.
    
    """
    participants = df["participant"].unique()
    participant_results = []
    all_preds, all_true_values, all_test_participants = [], [], []
    
    # Grid Search
    best_score = -float("inf")
    best_estimator = None
    
    # Define inner CV
    inner_cv = GroupKFold(n_splits=5) 
    if grid_search:
        for test_pid in participants:
            train_data, test_data = get_train_test_data(df, gt_full_set, test_pid, "Valence")
            if train_data is None or test_data is None:
                continue
    
            X_train = train_data.drop(columns=["participant", "video", "Valence"])
            y_train = train_data["Valence"].values
    
            groups_train = train_data["participant"]
            grid = GridSearchCV(
                LGBMRegressor(),
                param_grid,
                cv=inner_cv.split(X_train, y_train, groups_train),
                scoring='r2',
                n_jobs=-1
            )
            grid.fit(X_train, y_train, groups=groups_train)
    
            if grid.best_score_ > best_score:
                best_score = grid.best_score_
                best_estimator = grid.best_estimator_
    
        model_cls = best_estimator
        model_kwargs = best_estimator.get_params()
        
    for test_pid in participants:
        train_data, test_data = get_train_test_data(df, gt_full_set, test_pid, "Valence")
        if train_data is None or test_data is None:
            continue

        # Extract features and labels
        X_train = train_data.drop(columns=["participant", "video", "Valence"])
        y_train = train_data["Valence"].values
        X_test = test_data.drop(columns=["participant", "video", "Valence"])
        y_test = test_data["Valence"].values

        # Sequential Forward Selection (optional)
        if feature_selection:
            available_features = list(X_train.columns)
            selected_features = []
            best_score = -np.inf
            improved = True

            while improved and available_features:
                scores = []
                for feat in available_features:
                    try_features = selected_features + [feat]
                    estimator = clone(sfs_estimator_cls())
                    estimator.fit(X_train[try_features], y_train)
                    preds = estimator.predict(X_train[try_features])
                    score = scoring_func(y_train, preds)
                    scores.append((score, feat))
                scores.sort(reverse=True, key=lambda x: x[0])
                top_score, top_feature = scores[0]

                if top_score > best_score:
                    selected_features.append(top_feature)
                    available_features.remove(top_feature)
                    best_score = top_score
                    if max_features and len(selected_features) >= max_features:
                        break
                else:
                    improved = False

            X_train = X_train[selected_features]
            X_test = X_test[selected_features]

        # Train the final model
        model = model_cls(**deepcopy(model_kwargs))
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        # Evaluate
        fusion_metrics = evaluate_model(y_test, preds)
        fusion_metrics["participant"] = test_pid

        participant_results.append(fusion_metrics)
        all_preds.extend(preds)
        all_true_values.extend(y_test)
        all_test_participants.extend([test_pid] * len(y_test))

    return participant_results, all_preds, all_true_values, all_test_participants


**Linear Regression**

In [43]:
print("Linear Regression --------------- Valence")

model_cls = LinearRegression
model_kwargs = {}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=40
)

results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

Linear Regression --------------- Valence
       NRMSE         R2      corr             p participant
0   0.217893   0.618553  0.870690  5.142021e-12       F5tXL
1   0.138095   0.846784  0.932833  1.225714e-16        Gax8
2   0.222741   0.601389  0.892671  2.580932e-13       Hx7dO
3   0.177177   0.747789  0.897125  1.300662e-13       wohkw
4   0.196302   0.690403  0.873342  3.693082e-12       CVgs2
5   0.214205   0.636797  0.915332  1.379926e-14       G4dLl
6   0.189624   0.720169  0.910549  4.885029e-13       GFGzP
7   0.176830   0.748777  0.883733  9.363801e-13       PPjCX
8   0.175131   0.753581  0.926165  5.815981e-16       tMGNS
9   0.146017   0.828702  0.911430  1.142314e-14       5KB3V
10  0.223042   0.600315  0.865932  9.147837e-12       LR96S
11  0.154898   0.807230  0.903198  4.850837e-14        gf5X
12  0.204899   0.662693  0.852949  3.965785e-11       gjGzb
13  0.153894   0.809721  0.904979  3.587430e-14      A5feTy
14  0.197442   0.686797  0.861063  1.613307e-11       CoQm

**Ridge Regression**

In [44]:
print("Ridge Regression --------------- Valence")

model_cls = Ridge
model_kwargs = {}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=40
)

results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

Ridge Regression --------------- Valence


D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.387e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.38961e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.39405e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.44555e-21): result may not be accurate.
  return linalg

       NRMSE        R2      corr             p participant
0   0.239906  0.537587  0.864780  1.048297e-11       F5tXL
1   0.135989  0.851421  0.931502  1.693479e-16        Gax8
2   0.191270  0.706073  0.904045  4.205249e-14       Hx7dO
3   0.183489  0.729501  0.896786  1.371801e-13       wohkw
4   0.172788  0.760132  0.885016  7.833211e-13       CVgs2
5   0.229624  0.582628  0.898160  2.548326e-13       G4dLl
6   0.208874  0.660468  0.898624  2.941739e-12       GFGzP
7   0.168083  0.773016  0.882452  1.116636e-12       PPjCX
8   0.174984  0.753996  0.917688  3.453188e-15       tMGNS
9   0.148090  0.823803  0.911436  1.141076e-14       5KB3V
10  0.235556  0.554208  0.871025  4.933372e-12       LR96S
11  0.153342  0.811083  0.903926  4.290691e-14        gf5X
12  0.188146  0.715595  0.868712  6.551253e-12       gjGzb
13  0.151745  0.814999  0.916336  4.506742e-15      A5feTy
14  0.196407  0.690072  0.851362  4.698915e-11       CoQmx
15  0.173776  0.757379  0.877275  2.229217e-12      G5Xz

D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.37702e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
D:\MASTER\Uni of Essex\Disseration\Machine Learning\venv\lib\site-packages\sklearn\linear_model\_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.38648e-21): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


**Gradient Boosting Regressor**

In [45]:
print("GradientBoostingRegressor --------------- Valence")

model_cls = GradientBoostingRegressor
model_kwargs = {}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=40
)

results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

GradientBoostingRegressor --------------- Valence
       NRMSE        R2      corr             p participant
0   0.184022  0.727925  0.873967  3.412375e-12       F5tXL
1   0.091558  0.932650  0.984921  1.711737e-27        Gax8
2   0.089671  0.935396  0.984272  3.483521e-27       Hx7dO
3   0.135115  0.853325  0.970855  1.127020e-22       wohkw
4   0.089816  0.935189  0.967310  7.716513e-22       CVgs2
5   0.181003  0.740666  0.942632  2.754161e-17       G4dLl
6   0.093062  0.932601  0.976519  1.474538e-21       GFGzP
7   0.074966  0.954848  0.978161  8.816522e-25       PPjCX
8   0.113848  0.895864  0.974249  1.409427e-23       tMGNS
9   0.102942  0.914859  0.963398  5.116400e-21       5KB3V
10  0.125358  0.873744  0.971537  7.575156e-23       LR96S
11  0.116096  0.891711  0.946840  2.563756e-18        gf5X
12  0.151398  0.815844  0.963230  5.524006e-21       gjGzb
13  0.104023  0.913062  0.966054  1.450845e-21      A5feTy
14  0.134529  0.854596  0.928947  3.093816e-16       CoQmx
15  0.

**SVR**

In [46]:
print("SVR -------------- Valence")

model_cls = SVR
model_kwargs = {'kernel':'rbf', 'C':1.0, 'epsilon':0.1}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_arousal(
    df,
    gt_full_set,
    model_cls=model_cls,
    model_kwargs=model_kwargs,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,
    scoring_func=r2_score,
    max_features=40
)

results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

SVR -------------- Valence
       NRMSE        R2      corr         p participant
0   0.365046 -0.070636 -0.002454  0.988667       F5tXL
1   0.355729 -0.016686 -0.081268  0.637510        Gax8
2   0.356304 -0.019974  0.289744  0.086511       Hx7dO
3   0.354049 -0.007102 -0.041514  0.810024       wohkw
4   0.358967 -0.035276 -0.142910  0.405704       CVgs2
5   0.369835 -0.082690  0.088789  0.612008       G4dLl
6   0.361512 -0.017081 -0.120221  0.512212       GFGzP
7   0.357252 -0.025410  0.151212  0.378680       PPjCX
8   0.361109 -0.047666  0.077765  0.652131       tMGNS
9   0.356755 -0.022558 -0.022930  0.894396       5KB3V
10  0.363798 -0.063329  0.052695  0.760194       LR96S
11  0.356871 -0.023223 -0.195895  0.252200        gf5X
12  0.356396 -0.020498 -0.064896  0.706893       gjGzb
13  0.355667 -0.016327  0.052557  0.760806      A5feTy
14  0.358662 -0.033515 -0.130863  0.446807       CoQmx
15  0.356079 -0.018686  0.059273  0.731307      G5XzoP
16  0.372733 -0.116203 -0.164638  0.33

**LGBM**

In [47]:
print("LGBM -------------- Valence")
model_cls = LGBMRegressor
model_kwargs = {'num_leaves':31, 'max_depth':10, 'learning_rate':0.1, 'n_estimators':200}

participant_results, all_preds, all_true_values, all_test_participants = run_participant_loop_sfs_valence(
    df,
    gt_full_set,
    model_cls=model_cls,  # now a callable that returns a fitted model
    model_kwargs=model_kwargs,
    feature_selection=False,
    sfs_estimator_cls=LinearRegression,  # optional
    scoring_func=r2_score,
    max_features=40
)
results_df = pd.DataFrame(participant_results)
print(results_df)
print_summary(results_df)
print_filtered_summary(results_df)

LGBM -------------- Valence
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005644 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 85191
[LightGBM] [Info] Number of data points in the train set: 1575, number of used features: 344
[LightGBM] [Info] Start training from score -0.002535
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt